# Twitter User Details in Python

This notebook uses the Tweepy library for Python to access Twitter's streaming API to get tweets in real-time. The notebook demonstrates how to access user details for an individual account or list of accounts.

In order to use the Twitter Streaming API you will need to [Apply for a Twitter Developer Account](https://developer.twitter.com/en/application/use-case). You will then need to set up a Twitter appand obtain the authentication credentials that will enable your application to connect to the API. See [Twitter developer apps: Overview](https://developer.twitter.com/en/docs/basics/apps/overview) for further details.

#### Import Dependencies

In [ ]:
# Import Tweepy library to connect to the Twitter API and process the response
import tweepy
# Import Pandas to work with dataframes
import pandas as pd
# Import wget to retrieve profile images
import wget
# Import OS to manipulate file paths and names
import os
# Import Time to handle API rate limiting
import time

#### Authentication Details

In [ ]:
# Enter the details from the Twitter App you created here
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

#### Authentication

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth) 

## Get User's Profile Details

The `get_user()` method accepts the user's ID or screen name.

#### Specify user ID

In [ ]:
# Using screen name
_id = 'virt_arch'

#### Call the Twitter API

In [ ]:
user = api.get_user(_id)

#### Load response into a dataframe

In [ ]:
df = pd.DataFrame({'user_id': [user.id_str],
                   'screen_name': [user.screen_name],
                   'location': [user.location],
                   'profile_image': [user.profile_image_url],
                   'description': [user.description],
                   'website': [user.entities['url']['urls'][0]['expanded_url']],
                   'followers': [user.followers_count], 
                   'friends': [user.friends_count],
                   'created_at': [user.created_at], 
                   'verified': [user.verified]
                  })

#### Display the dataframe

In [ ]:
df

#### Download and save the user's profile picture

In [ ]:
# Remove the '_normal' tag from the URL to get a full sized image
link = user.profile_image_url.replace('_normal','')

# Set the ouput location for the image and save as png
destination = os.path.join('logos/' + user.screen_name + '.png')

# Use wget to download and save the image file
wget.download(link, destination)

#### Use HERE Maps API to geocode the user's location

In [ ]:
import herepy

geocoderApi = herepy.GeocoderApi('')

response = geocoderApi.free_form(user.location)

#### Display the returned coordinates

In [ ]:
print(response.items[0]['position'])

#### Append coordinates to the user dataframe

In [ ]:
df['lon'] = [response.items[0]['position']['lng']]
df['lat'] = [response.items[0]['position']['lat']]

#### Display the dataframe

In [ ]:
df

## Get User's Followers

The `followers()` method accepts the user's ID or screen name. We will use the `Cursor()` method to iterate through items. The number of items can be specific in `items()` or else left blank to iteract through all items. 

This request is subject to [rate limiting](https://developer.twitter.com/ja/docs/basics/rate-limits) against the 'Users' endpoint. It is recommended to set the rate limit of several seconds to avoid exceeding that rate limit.

#### Set time for rate limiter (seconds)

In [ ]:
rate_limit = 5

#### Specify User

In [ ]:
# Using screen name
_id = 'virt_arch'

#### Create Empty Dataframe

In [ ]:
df = pd.DataFrame(columns=['user_id',
                           'screen_name',
                           'location',
                           'profile_image',
                           'description',
                           'website',
                           'followers',
                           'friends',
                           'created_at',
                           'verified'
                          ])

#### Call the Twitter API

In [ ]:
for user in tweepy.Cursor(api.followers, _id).items():
    temp_df = pd.DataFrame({'user_id': [user.id_str],
                            'screen_name': [user.screen_name],
                            'location': [user.location],
                            'profile_image': [user.profile_image_url],
                            'description': [user.description],
                            'website': [user.entities['url']['urls'][0]['expanded_url']],
                            'followers': [user.followers_count], 
                            'friends': [user.friends_count],
                            'created_at': [user.created_at], 
                            'verified': [user.verified]
                           })
    df = df.append(temp_df)
    
    try:
        # Remove the '_normal' tag from the URL to get a full sized image
        link = user.profile_image_url.replace('_normal','')

        # Set the ouput location for the image and save as png
        destination = os.path.join('logos/' + user.screen_name + '.png')

        # Use wget to download and save the image file
        wget.download(link, destination)
    except:
        print(user.screen_name + ' profile image not available')
    
    print(df)
    time.sleep(rate_limit)

#### Display Dataframe

In [ ]:
df

#### Save Dataframe to CSV

In [ ]:
df.to_csv(os.path.join('data/' + _id + '_' + 'followers.csv'))